In [24]:
import os
from os.path import join, exists
import imageio as iio
import numpy as np

This notebook uses ffmpeg -- if you don't have it installed, go get it first.

In [25]:
# start: hh:mm:ss
# to: hh:mm:ii
# input / output vids: full paths including extensions
trim_command = 'ffmpeg -ss {start} -i {input_vid} -to {end} -c:v copy {output_vid}'

start = '00:00:00'
end = '00:02:00'
input_dir = '/n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221011_gmou83/'  # abs path
input_vid_name = '20221011_gmou83'  # relative prefix
output_dir = '/n/groups/datta/Jonah/gh_topbottom/20221011_validation'  # abs path
output_suffix = '_TRIM' 

input_vid_paths = [f.path for f in os.scandir(input_dir) if '.avi' in f.path]
input_vid_paths.sort()
output_vids_paths = [join(output_dir, input_vid_name + output_suffix + '.' + cam + '.' + stream + '.avi') for cam in ['bottom', 'top'] for stream in ['depth', 'ir']]

In [26]:
for in_vid, out_vid in zip(input_vid_paths, output_vids_paths):
    os.system(trim_command.format(start=start, end=end, input_vid=in_vid, output_vid=out_vid))

ffmpeg version 3.3.3-static http://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 6.4.0 (Debian 6.4.0-2) 20170724
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc-6 --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gray --enable-libass --enable-libfreetype --enable-libfribidi --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-librtmp --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libzimg
  libavutil      55. 58.100 / 55. 58.100
  libavcodec     57. 89.100 / 57. 89.100
  libavformat    57. 71.100 / 57. 71.100
  libavdevice    57.  6.100 / 57.  6.100
  libavfilter     6. 82.100 /  6. 82.100
  libswscale   

In [29]:
timestamp_paths = [f.path for f in os.scandir(input_dir) if '.npy' in f.path and 'matched' not in f.path]
timestamp_paths.sort()
output_ts_paths = [join(output_dir, input_vid_name + output_suffix + '.' + cam + '.' + stream + '.npy') for cam in ['bottom', 'top'] for stream in ['device_timestamps', 'system_timestamps']]
for out_vid, ts_in_path, ts_out_path in zip(output_vids_paths, timestamp_paths, output_ts_paths):
    nframes = iio.get_reader(out_vid).count_frames()  # should be quick since they're short videos
    all_ts = np.load(ts_in_path)
    np.save(ts_out_path, all_ts[0:nframes])
    